### Imports

In [11]:
import pandas as pd
import numpy as np
from numpy import nan as NAd

# Chapter 7. Data Cleaning and Preperation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language, like Python, Perl, R, or Java, or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form. 

If you identify a type of data manipulation that isn’t anywhere in this book or elsewhere in the pandas library, feel free to share your use case on one of the Python mailing lists or on the pandas GitHub site. Indeed, much of the design and implementation of pandas has been driven by the needs of real-world applications. In this chapter I discuss tools for missing data, duplicate data, string manipulation, and some other analytical data transformations. In the next chapter, I focus on combining and rearranging datasets in various ways.

## Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default. 

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [7]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [9]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data. 

The built-in Python *None* value is also treated as NA in object arrays:

In [10]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful. On a Series, it returns the Series with only the non-null data and index values:

In [13]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. dropna by default drops any row containing a missing value:

In [18]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing how =' all' will only drop rows that are all NA:

In [19]:
data.dropna(how = 'all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
to drop columns:

In [20]:
data.dropna(axis = 1, how = 'all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Another related way to filer out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observatins. You can indicate this with the thresh argument:

In [22]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.432300,NaN,NaN
1,0.176293,NaN,NaN
2,-0.714123,NaN,-0.636126
3,-0.760058,NaN,-0.571360
4,-1.428529,-0.743270,-1.458498
5,0.644434,-0.474857,-1.220513
6,-1.090696,-0.456390,0.916485


In [23]:
df.dropna()

,0,1,2
4,-1.428529,-0.743270,-1.458498
5,0.644434,-0.474857,-1.220513
6,-1.090696,-0.456390,0.916485


In [24]:
df.dropna(thresh = 2)

,0,1,2
2,-0.714123,NaN,-0.636126
3,-0.760058,NaN,-0.571360
4,-1.428529,-0.743270,-1.458498
5,0.644434,-0.474857,-1.220513
6,-1.090696,-0.456390,0.916485


### Filling in Missing Data

Rather than filtering put missing data (and potentially discarding other data along with it), you may want to fill in the "holes" in any number of ways. For most purposes, the fillna method is the workhorse function to use. Calling fillna with a constant replaces missing values with that value:

In [25]:
df.fillna(0)

,0,1,2
0,-0.432300,0.000000,0.000000
1,0.176293,0.000000,0.000000
2,-0.714123,0.000000,-0.636126
3,-0.760058,0.000000,-0.571360
4,-1.428529,-0.743270,-1.458498
5,0.644434,-0.474857,-1.220513
6,-1.090696,-0.456390,0.916485


In [26]:
df.fillna({1: 0.5, 2 : 0})

,0,1,2
0,-0.432300,0.500000,0.000000
1,0.176293,0.500000,0.000000
2,-0.714123,0.500000,-0.636126
3,-0.760058,0.500000,-0.571360
4,-1.428529,-0.743270,-1.458498
5,0.644434,-0.474857,-1.220513
6,-1.090696,-0.456390,0.916485


## Data Transformation

### Removing duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [27]:
data = pd.DataFrame({'k1' : ['one', 'two'] * 3 + ['two'],
                     'k2' : [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [28]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [29]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [30]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


### Transforming data using a function or mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [43]:
data = pd.DataFrame({'food' : ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon',
                               'pastrami', 'honey ham', 'nova lox'],
                     'ounces' : [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you want to add a columnd indicating the type of animal that each food came from. Let's write down a mapping of each distinct meat type to the kind of animal:

In [44]:
meat_to_animal = {
    'bacon' : 'pig',
    'pulled pork' : 'pig',
    'pastrami' : 'cow',
    'corned beef' : 'cow',
    'honey ham' : 'pig',
    'nova lox' : 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not. Thus, we need to convert each value to lowercase using the str.lower Series method:

In [45]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [46]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement. As you've already seen, map can be used to modify a subset of values in an object, but replace provides a simpler and more flexible way to do so. Let's consider this Series:

In [49]:
data = pd.Series([1., -999, 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [50]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [51]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [52]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [53]:
data.replace({-999 : np.nan, -1000 : 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

Like values in Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here's a simple example:

In [58]:
data = pd.DataFrame(
    np.arange(12).reshape((3, 4)),
    index = ['Ohio', 'Colorado', 'New York'],
    columns = ['one', 'two', 'three', 'four']
)

transform = lambda x : x[:4].upper()

data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

Continious data is often discretized or otherwise seperated into "bins" for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [59]:
ages = [20, 22, 25, 27, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let's divide these into bins of 18 to 25, 25 to 35, 36 to 60 and 61 and older.

In [60]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (25, 35], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 13
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

Pandas returns a Categorical object. The output you see describes the bins computed by pandas.cut. You can treat it like an array of strings indicating the bin name; internally it contains a categories array specifying the distinct category names along with a labeling for the ages data in the codes attribute

In [61]:
cats.codes

array([0, 0, 0, 1, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [62]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [63]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     4
(35, 60]     3
(60, 100]    1
dtype: int64

Note that pd.value_counts(cats) are the bin counts for the result of pandas.cut 

Consisten with mathematical notation for intervals, a parenthesis means that the side is *open*, while the square bracket means it is closed (inclusive), you can change which side is closed by passing right=False

In [64]:
pd.cut(ages, [18, 26, 36, 61, 100], right = False)

[[18, 26), [18, 26), [18, 26), [26, 36), [26, 36), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 13
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the labels option:

In [65]:
group_names = ['Youth', 'Young Adult', 'Middle Aged', 'Senior']
pd.cut(ages, bins, labels = group_names)

['Youth', 'Youth', 'Youth', 'Young Adult', 'Young Adult', ..., 'Young Adult', 'Senior', 'Middle Aged', 'Middle Aged', 'Young Adult']
Length: 13
Categories (4, object): ['Youth' < 'Young Adult' < 'Middle Aged' < 'Senior']

If you pass an integer number of bins to cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maxoimum values in the data.

Consider the case of some uniformly distributed data chopped into fourths:

In [67]:
data = np.random.randn(20)
pd.cut(data, 4, precision = 2)

[(-2.17, -0.92], (-2.17, -0.92], (1.57, 2.81], (0.32, 1.57], (-2.17, -0.92], ..., (0.32, 1.57], (-2.17, -0.92], (-2.17, -0.92], (0.32, 1.57], (0.32, 1.57]]
Length: 20
Categories (4, interval[float64]): [(-2.17, -0.92] < (-0.92, 0.32] < (0.32, 1.57] < (1.57, 2.81]]

A closely related function is qcut, which bins the data based on sample quantiles. Depending on the distribution of the data, using *cut* will not usually result in each bin having the same number of data points. Since *qcut* uses sample quantiles instead, by definition, you will obtain roughly equal-size bins:

In [69]:
data = np.random.randn(100)
cats = pd.qcut(data, 5)
cats

[(-0.162, 0.336], (0.883, 2.69], (0.336, 0.883], (-0.695, -0.162], (-2.4579999999999997, -0.695], ..., (0.883, 2.69], (0.883, 2.69], (-2.4579999999999997, -0.695], (-2.4579999999999997, -0.695], (-0.695, -0.162]]
Length: 100
Categories (5, interval[float64]): [(-2.4579999999999997, -0.695] < (-0.695, -0.162] < (-0.162, 0.336] < (0.336, 0.883] < (0.883, 2.69]]

In [70]:
pd.value_counts(cats)

(0.883, 2.69]                    20
(0.336, 0.883]                   20
(-0.162, 0.336]                  20
(-0.695, -0.162]                 20
(-2.4579999999999997, -0.695]    20
dtype: int64

You can also pass your own quantiles

In [71]:
pd.qcut(data, [0., 0.1, 0.5, 0.9, 1.])

[(-1.036, 0.0971], (0.0971, 1.143], (0.0971, 1.143], (-1.036, 0.0971], (-2.4579999999999997, -1.036], ..., (0.0971, 1.143], (1.143, 2.69], (-1.036, 0.0971], (-2.4579999999999997, -1.036], (-1.036, 0.0971]]
Length: 100
Categories (4, interval[float64]): [(-2.4579999999999997, -1.036] < (-1.036, 0.0971] < (0.0971, 1.143] < (1.143, 2.69]]

### Detecting and filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [72]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.010927,0.073336,-0.035053,0.012853
std,0.983860,1.005231,1.011859,0.989780
min,-3.506544,-2.799787,-3.356892,-3.902563
25%,-0.673895,-0.641420,-0.712553,-0.629876
50%,0.036989,0.080864,-0.028653,0.033141
75%,0.711912,0.769657,0.619613,0.677755
max,2.748324,3.450485,2.890089,2.676462


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [75]:
col = data[2]
col[np.abs(col) > 3]

493   -3.356892
Name: 2, dtype: float64

To select all rows having a value exceeding 3 og -3, you can use the *any* method on boolean DataFrame:

In [76]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
18,-1.490624,0.308185,1.642902,-3.462517
38,-0.613676,-1.836984,0.752027,-3.277380
245,0.110977,2.323161,0.140648,-3.005600
416,-1.394831,3.450485,1.115652,-0.105470
493,-1.720816,-1.413645,-3.356892,-0.236287
623,0.360417,0.917013,0.597380,-3.902563
662,-3.506544,1.216323,-0.466800,-1.412257
909,1.904182,0.502468,-0.381324,-3.082525


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the numpy.random.permutation function. Calling *permutation* with the length of the acis you want to permute produces an array of integers indicating the new ordering:

In [82]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([1, 3, 4, 0, 2])

In [83]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [84]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19
0,0,1,2,3
2,8,9,10,11


To select a random subset without replacement, you can use the sample method on Series and Dataframe.

In [85]:
df.sample(n = 3)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15


To generate a sample *with* replacement (to allow repeat choices), pass replace=True to sample:

In [86]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n = 10, replace = True)
draws

0    5
3    6
4    4
2   -1
3    6
2   -1
0    5
1    7
2   -1
3    6
dtype: int64

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a "dummy" or "indicator" matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s pandas has a get_dummies function for doing this.

In [87]:
df = pd.DataFrame(
    {'key' : ['b', 'b', 'a', 'c', 'a', 'b'],
     'data1' : range(6)}
)

pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0
